# __ctcf-footprint workflow__

1. find CTCF motifs with fimo
<br>```run-find-motifs-with-fimo.sh```
2. intersect Merged_CTCF_motifs.bed with CTCF-ChIP-peaks-and-DNase-hotspot.bed
<br>```bedtools intersect -u -f 1 -a Merged_CTCF_motifs.bed -b CTCF-ChIP-peaks-and-DNase-hotspot.bed > CTCF_candidate_footprints.bed```
3. separate by motif type
<br>```grep CTCF_(M,L,XL) CTCF_candidate_footprints.bed > CTCF_candidate_footprints_(M,L,XL).bed```
4. get fiber-seq reads around CTCF motifs
<br>```ft center GM12878.aligned.bam <(cut -f 1,2,3,6 CTCF_candidate_footprints_(M,L,XL).bed) > CTCF_m6a_fiberseq_(M,L,XL).txt```

future work
* filter by p-value
* get negative control (use same data but inverse)
* --> m6a methylation NOT within ChIP or DNase I peaks